In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "4,5"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

from trl import SFTTrainer, SFTConfig

In [3]:
model_name = "Behzadshomali/Teuken3.7B"
device = "cuda:0"

base_tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
base_model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)

Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.71it/s]


In [4]:
sft_model_name = "Behzadshomali/Teuken3.7B_IT_LoRA"
revision = "22_04_11_checkpoint-1125"

sft_tokenizer = AutoTokenizer.from_pretrained(sft_model_name, trust_remote_code=True, revision=revision)
sft_model = AutoModelForCausalLM.from_pretrained(sft_model_name, trust_remote_code=True, revision=revision)

Loading checkpoint shards: 100%|██████████| 8/8 [00:00<00:00, 327.45it/s]


In [5]:
from transformers import pipeline

In [43]:
# question = "James decides to run 3 sprints 3 times a week. He runs 60 meters each sprint. How many total meters does he run a week?"
generator_base = pipeline(
    "text-generation", 
    base_model, 
    tokenizer=base_tokenizer, 
    device_map="auto",
    temperature=.7
)

generator_sft = pipeline(
    "text-generation", 
    sft_model, 
    tokenizer=sft_tokenizer, 
    device_map="auto",
    temperature=.3
)

Device set to use cuda:0
The model 'GPT2ForCausalLM' is not supported for text-generation. Supported models are ['PeftModelForCausalLM', 'ArceeForCausalLM', 'AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BitNetForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV2ForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'DogeForCausalLM', 'Dots1ForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'Ernie4_5ForCausalLM', 'Ernie4_5_MoeForCausalLM', 'Exaone4ForCausalLM', 'FalconForCausalLM', 'FalconH1ForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM'

In [44]:
question2 = "Marissa is hiking a 12-mile trail. She took 1 hour to walk the first 4 miles, then another hour to walk the next two miles. If she wants her average speed to be 4 miles per hour, what speed (in miles per hour) does she need to walk the remaining distance?"

In [45]:
output_base = generator_base([question2], max_new_tokens=512, return_full_text=False)[0]
output_sft = generator_sft([{"role": "user", "content": question2}], max_new_tokens=512, return_full_text=False)[0]

In [ ]:
print("BASE:\n", output_base[0]['generated_text'])
print("="*20)
print("SFT:\n", output_sft['generated_text'])

BASE:
 

Marissa 12 miles of trail in 2 hours. If she wants her average speed to be 4 miles per hour, what speed (in miles per hour) does she need to walk the remaining distance?

What is the answer?

I am stuck on this question. Can someone please help me?

👍

👎

ℹ️

🚩

Anonymous

Aug 27, 2016

12 miles (total) / 2 hours (total) = 6 miles per hour.

👍

👎

ℹ️

🚩

Ms. Sue

Aug 27, 2016

3 miles per hour

👍

👎

ℹ️

🚩

Ms. Sue

Aug 27, 2016

3 miles per hour

👍

👎

ℹ️

🚩

Ms. Sue

Aug 27, 2016

3 miles per hour

👍

👎

ℹ️

🚩

Ms. Sue

Aug 27, 2016

3 miles per hour, 4 miles per hour

👍

👎

ℹ️

🚩

Ms. Sue

Aug 27, 2016

3 miles per hour, 4 miles per hour

👍

👎

ℹ️

🚩

Ms. Sue

Aug 27, 2016

3 miles per hour, 4 miles per hour

👍

👎

ℹ️

🚩

Ms. Sue

Aug 27, 2016

3 miles per hour, 4 miles per hour

👍

👎

ℹ️

🚩

Ms. Sue

Aug 27, 2016

3 miles per hour, 4 miles per hour

👍

👎

ℹ️

🚩

Ms
SFT:
 

/

 How many miles does Marissa walk in the first 2 hours? ** Marissa walks 4+2=<<4+2=6>>6 miles in t

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3,4"

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from trl import clone_chat_template

# Base model (same one you used during training)
base_model = "Behzadshomali/Teuken3.7B"

# Your LoRA model (local folder or HF Hub repo)
lora_model = "/raid/s3/opengptx/behzad_shomali/instruction_tuning/Teuken3.73T_IT_GSM8K_socratic/17_22_56/checkpoint-675/"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map="auto",
    trust_remote_code=True
)

model, tokenizer, _ = clone_chat_template(model, tokenizer, "Qwen/Qwen3-0.6B")

# Load LoRA on top of base
model = PeftModel.from_pretrained(model, lora_model)

# Merge LoRA adapters into the base model
model = model.merge_and_unload()


revision = "22_04_11_checkpoint-1125"
# model.push_to_hub(
#     "Behzadshomali/Teuken3.7B_IT_LoRA",
#     revision=revision,
#     private=True,             # optional
#     use_temp_dir=True,        # makes sure full files are synced
#     commit_message="Add merged LoRA model with custom code",
#     safe_serialization=True
# )

tokenizer.push_to_hub("Behzadshomali/Teuken3.7B_IT_LoRA", revision=revision)

/raid/s3/opengptx/behzad_shomali/miniforge3/envs/HF/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.88s/it]
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/Behzadshomali/Teuken3.7B_IT_LoRA/commit/008e5b23f1bbcaa046bfe2d848fa435539cb11e7', commit_message='Upload tokenizer', commit_description='', oid='008e5b23f1bbcaa046bfe2d848fa435539cb11e7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Behzadshomali/Teuken3.7B_IT_LoRA', endpoint='https://huggingface.co', repo_type='model', repo_id='Behzadshomali/Teuken3.7B_IT_LoRA'), pr_revision=None, pr_num=None)

In [7]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "4,5"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

from trl import SFTTrainer, SFTConfig

generator = pipeline(
    "text-generation", 
    model, 
    tokenizer=tokenizer, 
    device_map="auto",
    temperature=.3
)

Device set to use cuda:0
The model 'GPT2ForCausalLM' is not supported for text-generation. Supported models are ['PeftModelForCausalLM', 'ArceeForCausalLM', 'AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BitNetForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV2ForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'DogeForCausalLM', 'Dots1ForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'Ernie4_5ForCausalLM', 'Ernie4_5_MoeForCausalLM', 'Exaone4ForCausalLM', 'FalconForCausalLM', 'FalconH1ForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM'

In [8]:
question2 = "Marissa is hiking a 12-mile trail. She took 1 hour to walk the first 4 miles, then another hour to walk the next two miles. If she wants her average speed to be 4 miles per hour, what speed (in miles per hour) does she need to walk the remaining distance?"

In [9]:
output = generator([{"role": "user", "content": question2}], max_new_tokens=512, return_full_text=True)[0]

In [10]:
from pprint import pprint

print(output['generated_text'][1]['content'])

- The total time she spent walking the first 4 miles was 1+1 = <<1+1=2>>2 hours
- The total time she spent walking the next two miles was 2+1 = <<2+1=3>>3 hours
- The total time she spent walking the entire trail was 2+3 = <<2+3=5>>5 hours
- The total distance she walked is 12 miles
- Her average speed is 4 miles per hour
- She needs to walk the remaining distance at 4+5 = <<4+5=9>>9 miles per hour
 #### 9.00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000


In [ ]:
import yaml
import os
from pathlib import Path
from transformers import TrainerCallback
import json
from datetime import datetime
import asyncio
import threading
import wandb
import process
def lighteval_async(checkpoint_path, cuda_devices="3,4"):
    output_dir = "/raid/s3/opengptx/behzad_shomali/evaluation_results/sft_intermediate_results/"
    multi_gpu_command = "--multi_gpu" if len(cuda_devices.split(',')) > 1 else ""

    command = f"""\
CUDA_VISIBLE_DEVICES={cuda_devices} accelerate launch \
    {multi_gpu_command} \
    --main_process_port 2000 \
    --num_processes {len(cuda_devices.split(','))} \
    --num_machines 1 \
    -m \
    lighteval accelerate \
    "model_name={checkpoint_path},trust_remote_code=True,use_chat_template=True" \
    "leaderboard|gsm8k|0|0,leaderboard|hellaswag|0|0" \
    "--max_samples 100" \
    --output-dir {output_dir} \
"""
    
    # # Run subprocess asynchronously
    # proc = asyncio.create_subprocess_shell(
    #     command,
    #     stdout=asyncio.subprocess.PIPE,
    #     stderr=asyncio.subprocess.PIPE,
    #     cwd=os.getcwd()
    # )

    # stdout, stderr = proc.communicate()

    # if proc.returncode != 0:
    #     print("Evaluation failed!", stderr.decode())
    #     return None

    # # Find latest results JSON
    # output_dir = Path(output_dir)
    # json_files = [f for f in output_dir.glob("*.json") if f.is_file()]

    # if not json_files:
    #     return None

    # json_files.sort(key=lambda f: f.stat().st_ctime, reverse=True)
    # results_file = json_files[0]

    # with open(results_file, "r") as f:
    #     results_dict = json.load(f)["results"]

    # final_results = {
    #     benchmark: v["qem"]
    #     for benchmark, v in results_dict.items()
    #     if benchmark != "all"
    # }

    # return final_results

/raid/s3/opengptx/behzad_shomali/miniforge3/envs/HF/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
lighteval_async("X")

AttributeError: 'coroutine' object has no attribute 'communicate'